<a href="https://colab.research.google.com/github/dudaholandah/PIBIC/blob/main/PIBIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [227]:
#!pip install unidecode
#!pip install umap-learn[plot]
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn import preprocessing
from pandas.core.frame import DataFrame
import plotly.express as px
from sklearn.manifold import TSNE
import re
import umap
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import Isomap
from sklearn.manifold import trustworthiness
import plotly.graph_objects as go

# Pre-Processing Data

In [228]:
data = pd.read_excel("table_veg.xlsx", sheet_name='Veganos')

In [229]:
data.head()

,Classificação,Nome do Produto,Ingredientes,Porção,Kcal,Carboidratos,Açúcares,Proteínas,Gorduras Totais,Gorduras Saturadas,...,Carboidratos.1,Açúcares.1,Proteínas.1,Gorduras Totais.1,Gorduras Saturadas.1,Fibras .1,Sódio.1,B12.1,calcium.1,zinc.1
0,CARNES 2,Carne Moída do Futuro,"water, texturizedsoyprotein, peaprotein e chic...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
1,CARNES 3,Carne Moída do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
2,FRANGOS 3,Frango do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
3,SUÍNOS 1,Linguiça do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",50.0,139.0,7.7,0.0,5.7,9.5,3.9,...,15.40,0.0,11.40,19.0,7.80,0.400,588.0,0.0,0.0,0.0
4,CARNES 1,Hamburguer de Soja Goshen,"texturizedsoyprotein, water, soyoil, modified...",50.0,118.0,2.8,0.0,7.0,9.0,1.3,...,5.60,0.0,14.00,18.0,2.60,4.000,688.0,0.0,0.0,0.0


### Separating data

In [230]:
data_ingredients = pd.DataFrame(data['Ingredientes'])
data_nutrients = data.drop(columns=['Ingredientes', 'Classificação', 'Nome do Produto'])
data_clasification = pd.DataFrame(data['Classificação'])
data_name = pd.DataFrame(data['Nome do Produto'])
label_data = data_clasification.join(data_name)

label_data.head()

,Classificação,Nome do Produto
0,CARNES 2,Carne Moída do Futuro
1,CARNES 3,Carne Moída do Futuro
2,FRANGOS 3,Frango do Futuro
3,SUÍNOS 1,Linguiça do Futuro
4,CARNES 1,Hamburguer de Soja Goshen


In [231]:
data_nutrients.head()

,Porção,Kcal,Carboidratos,Açúcares,Proteínas,Gorduras Totais,Gorduras Saturadas,Fibras,Sódio,B12,...,Carboidratos.1,Açúcares.1,Proteínas.1,Gorduras Totais.1,Gorduras Saturadas.1,Fibras .1,Sódio.1,B12.1,calcium.1,zinc.1
0,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
1,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
2,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
3,50.0,139.0,7.7,0.0,5.7,9.5,3.9,0.2,294.0,0.0,...,15.40,0.0,11.40,19.0,7.80,0.400,588.0,0.0,0.0,0.0
4,50.0,118.0,2.8,0.0,7.0,9.0,1.3,2.0,344.0,0.0,...,5.60,0.0,14.00,18.0,2.60,4.000,688.0,0.0,0.0,0.0


### Normalizing Nutrient Values (MinMax Scaler) 

In [232]:
X = data_nutrients.values
print(X)

[[ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 ...
 [ 40.   84.    5.2 ...   0.    0.    0. ]
 [ 10.   25.    3.3 ...   0.    0.    0. ]
 [  7.   28.    1.2 ...   0.    0.    0. ]]


In [233]:
attributes_dummies = data_nutrients.columns
normalize = preprocessing.MinMaxScaler()
xscaled = normalize.fit_transform(X)

nutrients_normalized = pd.DataFrame(xscaled,columns=attributes_dummies)
nutrients_normalized = nutrients_normalized.replace(np.nan,0)

nutrients_normalized.head()

,Porção,Kcal,Carboidratos,Açúcares,Proteínas,Gorduras Totais,Gorduras Saturadas,Fibras,Sódio,B12,...,Carboidratos.1,Açúcares.1,Proteínas.1,Gorduras Totais.1,Gorduras Saturadas.1,Fibras .1,Sódio.1,B12.1,calcium.1,zinc.1
0,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
1,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
2,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
3,0.125364,0.212121,0.171111,0.0,0.135714,0.193878,0.211957,0.002128,0.310782,0.0,...,0.1848,0.0,0.228,0.077551,0.265909,0.000511,0.453470,0.0,0.0,0.0
4,0.125364,0.174688,0.062222,0.0,0.166667,0.183673,0.070652,0.021277,0.363636,0.0,...,0.0672,0.0,0.280,0.073469,0.088636,0.005106,0.530591,0.0,0.0,0.0


### Normalizing Ingredients (One Hot Encode)

In [234]:
ingredients_normalized = pd.DataFrame()

for i in range(276):
  aux = data_ingredients.at[i,'Ingredientes'].split(",")
  for w in aux:
    w = unidecode(w.lower().lstrip().replace('.',''))
    ingredients_normalized.at[i,w] = 1

ingredients_normalized = ingredients_normalized.replace(np.nan,0)
ingredients_normalized.head()

,water,texturizedsoyprotein,peaprotein e chickpeaflour,vegetalfat,modifiedstarch,onion,meatflavoredcondiment,salt,sugar,powderedbeet,...,cloreto de potassio,calciumdisodium and citricacid,isolatedvegetableprotein,hydrolyzedvegetableprotein,soy,potassiumsorbate,peastarch,peaprotein and sodiumbicarbonate,isolatedriceprotein,chickpeas and goldenflaxseed
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing data

In [235]:
raw_data = pd.DataFrame(ingredients_normalized)
raw_data = pd.concat([raw_data,nutrients_normalized], axis=1)
raw_data = raw_data.replace(np.nan,0)

raw_data.head()

,water,texturizedsoyprotein,peaprotein e chickpeaflour,vegetalfat,modifiedstarch,onion,meatflavoredcondiment,salt,sugar,powderedbeet,...,Carboidratos.1,Açúcares.1,Proteínas.1,Gorduras Totais.1,Gorduras Saturadas.1,Fibras .1,Sódio.1,B12.1,calcium.1,zinc.1
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.1848,0.0,0.228,0.077551,0.265909,0.000511,0.453470,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0672,0.0,0.280,0.073469,0.088636,0.005106,0.530591,0.0,0.0,0.0


In [236]:
X = raw_data.values
y = label_data.values
all_data = label_data.join(raw_data)

for i in range(276):
  label_data.at[i,'Classificação'] = label_data['Classificação'][i].rstrip()
  all_data.at[i,'Classificação'] = all_data['Classificação'][i].rstrip()

print(all_data['Classificação'].value_counts())

LATICÍNIOS 1    80
LATICÍNIOS 3    45
CARNES 1        34
CARNES 3        18
FRANGOS 1       14
OVOS 1          14
FRANGOS 3       13
SUÍNOS 1        12
LATICÍNIOS 2    11
CARNES 2         9
PESCADOS 1       8
SUÍNOS 2         6
FRANGOS 2        5
PESCADOS 2       5
OVOS 2           2
Name: Classificação, dtype: int64


# TSNE

In [251]:
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(X)

X_tsne[1:4, :]
all_data['x'] = X_tsne[:,0]
all_data['y'] = X_tsne[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [252]:
data_ingredients = label_data.join(ingredients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(ingredients_normalized)

X_tsne[1:4, :]
data_ingredients['x'] = X_tsne[:,0]
data_ingredients['y'] = X_tsne[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [253]:
data_nutrients = label_data.join(nutrients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(nutrients_normalized)

X_tsne[1:4, :]
data_nutrients['x'] = X_tsne[:,0]
data_nutrients['y'] = X_tsne[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

# UMAP

In [254]:
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(X)

X_umap[1:4, :]
all_data['x'] = X_umap[:,0]
all_data['y'] = X_umap[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [255]:
data_ingredients = label_data.join(ingredients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(ingredients_normalized)

X_umap[1:4, :]
data_ingredients['x'] = X_umap[:,0]
data_ingredients['y'] = X_umap[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [256]:
data_nutrients = label_data.join(nutrients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(nutrients_normalized)

X_umap[1:4, :]
data_nutrients['x'] = X_umap[:,0]
data_nutrients['y'] = X_umap[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

# PCA

In [257]:
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)

X_pca[1:4, :]
all_data['x'] = X_pca[:,0]
all_data['y'] = X_pca[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [258]:
data_ingredients = label_data.join(ingredients_normalized)
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(ingredients_normalized)

X_pca[1:4, :]
data_ingredients['x'] = X_pca[:,0]
data_ingredients['y'] = X_pca[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [259]:
data_nutrients = label_data.join(nutrients_normalized)
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(nutrients_normalized)

X_pca[1:4, :]
data_nutrients['x'] = X_pca[:,0]
data_nutrients['y'] = X_pca[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

# ISOMAP

In [260]:
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(X)

X_isomap[1:4, :]
all_data['x'] = X_isomap[:,0]
all_data['y'] = X_isomap[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [261]:
data_ingredients = label_data.join(ingredients_normalized)
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(ingredients_normalized)

X_isomap[1:4, :]
data_ingredients['x'] = X_isomap[:,0]
data_ingredients['y'] = X_isomap[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

In [262]:
data_nutrients = label_data.join(nutrients_normalized)
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(nutrients_normalized)

X_isomap[1:4, :]
data_nutrients['x'] = X_isomap[:,0]
data_nutrients['y'] = X_isomap[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classificação', hover_data=['Nome do Produto'])
fig.show()

# Trustworthiness

### TSNE

In [263]:
kneigh = []
y_tsne = np.zeros((10,21))
y_pca = []
y_isomap = []

v_perplexity = [5,10,15,20,25,30,35,40,45,50]
v_learning_rate = [10,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]

for pv in range(0,21):
  kneigh.append(v_learning_rate[pv])
  for pi in range(0,10):
    tsne = TSNE(n_components=2,perplexity=v_perplexity[pi],learning_rate=v_learning_rate[pv],metric='euclidean', init='random')
    X_tsne = tsne.fit_transform(X)
    precision = trustworthiness(X, X_tsne, n_neighbors=1, metric='euclidean')
    y_tsne[pi][pv] = precision 

In [264]:
fig = go.Figure()

for pi in range(0,10):
  fig.add_trace(go.Scatter(x=kneigh, y=y_tsne[pi],
                    mode='lines+markers',
                    name='t-SNE perplexity ' + str(v_perplexity[pi])))

fig.show()

### UMAP

In [265]:
v_neighbor = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_umap = []

for n in v_neighbor:
  kneigh.append(n)
  reducer = umap.UMAP(n_components=2,n_neighbors=n,metric='euclidean')
  X_umap = reducer.fit_transform(X)
  precision = trustworthiness(X, X_umap, n_neighbors=1, metric='euclidean')
  y_umap.append(precision)

In [266]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                  mode='lines+markers',
                  name='UMAP'))

fig.show()

### ISOMAP

In [267]:
v_neighbor = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_isomap = []

for n in v_neighbor:
  kneigh.append(n)
  isomap = Isomap(n_components=2,n_neighbors=n,)
  X_isomap = isomap.fit_transform(X)
  precision = trustworthiness(X, X_isomap, n_neighbors=1, metric='euclidean')
  y_isomap.append(precision)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_isomap.py:324: UserWarning:

The number of connected components of the neighbors graph is 27 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [268]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                  mode='lines+markers',
                  name='UMAP'))

fig.show()

### Comparing Results

All Data (Ingredients + Nutrients)

In [269]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(X)

reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(X)

sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)

isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(X)

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsne, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umap, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pca, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomap, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [270]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

Only Ingredients

In [271]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

data_ingredients = label_data.join(ingredients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(ingredients_normalized)

data_ingredients = label_data.join(ingredients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(ingredients_normalized)

data_ingredients = label_data.join(ingredients_normalized)
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)

data_ingredients = label_data.join(ingredients_normalized)
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(X)

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsne, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umap, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pca, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomap, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [272]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

Only Nutrients

In [273]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

data_nutrients = label_data.join(nutrients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(nutrients_normalized)

data_nutrients = label_data.join(nutrients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(nutrients_normalized)

data_nutrients = label_data.join(nutrients_normalized)
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)

data_nutrients = label_data.join(nutrients_normalized)
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(X)

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsne, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umap, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pca, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomap, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [274]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()